# Quantum blockchain
1. What is the optimal way to create quantum key between the nodes of a classical network?

In [1]:
import networkx as nx 
import math
import pandas as pd
G = nx.Graph()
G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(2,3)
G.add_edge(2,4)
G.add_edge(3,5)
G.add_edge(4,5)
G.add_edge(4,6)
G.add_edge(5,6)
#nx.draw(G, with_labels=True)
num_edges = nx.number_of_edges(G)
param_matrix = [[0]*(num_edges+1) for i in range(num_edges+1)]
#define with [distance (km), coupling eff. (%), num_qubits, attenuation length (km)]
param_matrix[1][2] = [30, 0.95, 20*5, 20]
param_matrix[1][3] = [30, 0.95, 30*5, 20]
param_matrix[2][3] = [30, 0.8, 30*5, 20]
param_matrix[2][4] = [20, 0.9, 20*5, 20]
param_matrix[3][5] = [20, 0.80, 20*5, 20]
param_matrix[4][5] = [30, 0.77, 15*5, 20]
param_matrix[4][6] = [20, 0.83,25*5, 20]
param_matrix[5][6] = [30, 0.99, 30*5, 20]
#We could print the list of all paths between two nodes (1,6) in the network using the simple function

# Entanglement generation (nodes)
We could start with entangled link generation between two nodes. 
Strategy: Try to create entangled pairs between "m" qubits. in case, none of them suceed, try again (try "neg" times). 
Probability of getting one pair (prob) = (1-(1-p)**(neg*m))
time taken = neg*dist/c (km/s in fiber)

In [2]:
def elink_edges(G, i,j, param):
    dist = param[i][j][0]
    latt = param[i][j][3]
    pc = (param[i][j][1])#coupling efficiency
    p = 0.5*(pc**2)* math.exp(-dist/latt)
    num_pairs = math.floor(param[i][j][2]/len(list(G.nodes()))-1) #number of pairs (edges) for one elink
    rate = 0
    prob_max = 0
    for neg in range(1,5):
        prob = (1-(1-p)**(neg*num_pairs)) #probability
        time = neg*dist/(2*1e5)
        if prob > prob_max:
            rate = prob*(1/time)
            qubits = num_pairs
            prob_max = prob
            time_comm = time
            neg_max = neg
    return rate, qubits, prob_max, time_comm, neg_max


def rate(links, elink):
    t_max = 0
    prob = 1
    qubits = 0
    for var in range(len(links)-1):
        if elink[links[var]][links[var+1]] ==0:
            prob= prob*elink[links[var+1]][links[var]][2]
        else:
            prob= prob*elink[links[var]][links[var+1]][2]
        
        if elink[links[var]][links[var+1]] ==0 and elink[links[var+1]][links[var]][3]> t_max:
            time =  elink[links[var+1]][links[var]][3]
            qubits = qubits + elink[links[var+1]][links[var]][1]
        elif elink[links[var]][links[var+1]][3]> t_max:
            time =  elink[links[var]][links[var+1]][3]
            qubits = qubits + elink[links[var]][links[var+1]][1]
    
    return qubits/(prob/time), qubits, time, prob


## Optimization with cost function
The optimized path is obtained by defining a cost function and find the path that yields the minimum cost

In [3]:
def cost_function(node_a, node_b, num_edges):
    paths = list(nx.all_simple_paths(G, node_a, node_b, cutoff=None))
    cost = [0]*(len(paths))
    qubits_tot = [0]*(len(paths))
    time = [0]*(len(paths))
    p_success = [0]*(len(paths))
    elink = [[0]*num_edges]*num_edges
    for a in range(len(paths)):
        for b in range(len(paths[a])-1):
            node_1 = paths[a][b]
            node_2 = paths[a][b+1]
            if node_1 > node_2:
                node_1 = node_2
                node_2 = paths[a][b]
            elink[node_1][node_2]= elink_edges(G, node_1,node_2, param_matrix) 
    for i in range(len(paths)):
        cost[i], qubits_tot[i], time[i],p_success[i] = rate(paths[i], elink)
    #print("The optimum path to nodes",node_a, "and", node_b, "based on the cost function is" ,paths[cost.index(min(cost))])
    return node_a, node_b, paths[cost.index(min(cost))], qubits_tot[cost.index(min(cost))], time[cost.index(min(cost))], p_success[cost.index(min(cost))]
    

In [13]:
nodes = list(G.nodes())
output = []
for i in range(0, len(nodes)):
    for j in range(i+1,len(nodes)):
        node_a, node_b, path, qubits, time, psuccess = cost_function(nodes[i], nodes[j], num_edges)
        output.append([node_a, node_b, path, qubits, time, psuccess])
df = pd.DataFrame(output, columns=['Node_1', 'Node_2', 'optimized path', 'qubits', 'time(s)', 'P_success'])
df

,Node_1,Node_2,optimized path,qubits,time(s),P_success
0,1,2,"[1, 2]",15,0.0006,0.998284
1,1,3,"[1, 3]",24,0.0006,0.999962
2,1,4,"[1, 2, 4]",30,0.0004,0.998221
3,1,5,"[1, 3, 5]",39,0.0004,0.999417
4,1,6,"[1, 2, 4, 6]",54,0.0006,0.998206
5,2,3,"[2, 3]",24,0.0006,0.999184
6,2,4,"[2, 4]",15,0.0004,0.999937
7,2,5,"[2, 4, 5]",26,0.0006,0.950708
8,2,6,"[2, 4, 6]",34,0.0004,0.999904
9,3,4,"[3, 5, 4]",30,0.0004,0.999393
